# CasADi（Python）用法与 Demo

本 notebook 以“可运行”为目标，覆盖：
- 符号建模：`SX/MX`
- Function：封装与调用
- 自动求导：`jacobian/hessian/jtimes`
- NLP：`nlpsol(ipopt)` 最小例子
- ODE 积分器：`integrator(cvodes/rk)`
- QP：`qpsol`（带插件 fallback）
- Root-finding：`rootfinder`（最小例子）
- Opti（可选）：更贴近数学表达的建模方式

## 0. 环境检查

In [1]:
import casadi as ca
import numpy as np

print('casadi:', ca.__version__)
print('numpy :', np.__version__)

casadi: 3.7.2
numpy : 2.3.5


## 1. 符号变量与表达式：SX vs MX

- 小型/标量化表达：SX 更轻量
- 大型/含算子调用（integrator/solver）：MX 更通用

建议：先用 MX 写通，再考虑局部用 SX 优化性能。

In [2]:
# SX 示例
xs = ca.SX.sym('x')
ys = ca.SX.sym('y', 3)
expr_sx = ca.sqrt(xs**2 + 10) + ca.sum1(ys)
print('SX expr:', expr_sx)

# MX 示例
xm = ca.MX.sym('x', 2, 2)
ym = ca.MX.sym('y')
expr_mx = 3 * xm + ym
print('MX expr:', expr_mx)

SX expr: (sqrt((sq(x)+10))+((y_0+y_1)+y_2))
MX expr: ((3*x)+y)


## 2. Function：封装可复用算子

要点：给输入输出命名，后续用关键字调用。

In [3]:
x = ca.MX.sym('x', 2)
y = ca.MX.sym('y')
f = ca.Function('f', [x, y], [x, ca.sin(y) * x], ['x', 'y'], ['r', 'q'])

res = f(x=[1, 2], y=0.5)
print('r =', res['r'])
print('q =', res['q'])

r = [1, 2]
q = [0.479426, 0.958851]


## 3. 自动求导（AD）

演示：
- 显式 Jacobian / Hessian
- `jtimes` 做 $Jv$ 与 $J^T w$（方向导/反向导）

In [4]:
x = ca.SX.sym('x', 2)
f = ca.vertcat(x[0]**2 + ca.sin(x[1]), x[0]*x[1])
J = ca.jacobian(f, x)
H, g = ca.hessian(ca.dot(f, f), x)

print('f(x)=', f)
print('J=\n', J)
print('grad=\n', g)
print('H=\n', H)

v = ca.SX.sym('v', 2)
w = ca.SX.sym('w', 2)
Jv = ca.jtimes(f, x, v)           # J v
JT_w = ca.jtimes(f, x, w, True)  # J^T w
print('Jv=', Jv)
print('J^T w=', JT_w)

f(x)= [(sq(x_0)+sin(x_1)), (x_0*x_1)]
J=
 
[[(x_0+x_0), cos(x_1)], 
 [x_1, x_0]]
grad=
 @1=(x_0*x_1), @2=(@1+@1), @3=(sq(x_0)+sin(x_1)), @4=(@3+@3), [((x_1*@2)+((x_0+x_0)*@4)), ((x_0*@2)+(cos(x_1)*@4))]
H=
 @1=(sq(x_0)+sin(x_1)), @2=(@1+@1), @3=(x_0+x_0), @4=(x_0+x_0), @5=(x_0*x_1), @6=(x_0+x_0), @7=cos(x_1), @8=(@7+@7), @9=(((@5+@5)+(x_1*@6))+(@3*@8)), 
[[((x_1*(x_1+x_1))+((2*@2)+(@3*(@4+@4)))), @9], 
 [@9, ((x_0*@6)+((cos(x_1)*@8)-(@2*sin(x_1))))]]
Jv= [(((x_0+x_0)*v_0)+(cos(x_1)*v_1)), ((x_1*v_0)+(x_0*v_1))]
J^T w= [((x_1*w_1)+((x_0+x_0)*w_0)), ((x_0*w_1)+(cos(x_1)*w_0))]


## 4. NLP Demo：nlpsol + IPOPT（最小可跑例子）

Rosenbrock 形式的小例子：
- 决策变量 $(x,y,z)$
- 目标：$x^2 + 100 z^2$
- 等式约束：$z + (1-x)^2 - y = 0$

In [6]:
x = ca.SX.sym('x')
y = ca.SX.sym('y')
z = ca.SX.sym('z')

nlp = {
    'x': ca.vertcat(x, y, z),
    'f': x**2 + 100*z**2,
    'g': z + (1-x)**2 - y,
}
solver = ca.nlpsol('S', 'ipopt', nlp)
sol = solver(x0=[2.5, 3.0, 0.75], lbg=0, ubg=0)
print('x* =', sol['x'])
print('f* =', sol['f'])

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        2

Total number of variables............................:        3
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  6.2500000e+01 0.00e+00 9.00e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## 5. ODE 积分器 Demo：integrator

我们解一个最简单的指数衰减：
$$\dot{x} = -p x$$
从 $t=0$ 积分到 $t=1$。

说明：不同安装包可能 cvodes 插件可用性不同；此处提供 fallback 到 `rk`。

In [7]:
x = ca.SX.sym('x')
p = ca.SX.sym('p')
dae = {'x': x, 'p': p, 'ode': -p*x}

try:
    F = ca.integrator('F', 'cvodes', dae, 0.0, 1.0)
    print('Using plugin: cvodes')
except Exception as e:
    print('cvodes not available, fallback to rk:', e)
    F = ca.integrator('F', 'rk', dae, 0.0, 1.0)

for pval in [0.5, 1.0, 2.0]:
    r = F(x0=1.0, p=pval)
    print('p=', pval, ' x(1)=', float(r['xf']))

Using plugin: cvodes
p= 0.5  x(1)= 0.6065299541915267
p= 1.0  x(1)= 0.36787867582935213
p= 2.0  x(1)= 0.13533595525753633


## 6. Opti（可选）：更“像数学”的建模方式

Opti 会引入一定 overhead，但写起来更直观；需要高性能时可以用 `to_function()` 把大部分 overhead 去掉。

In [8]:
opti = ca.Opti()
x = opti.variable()
y = opti.variable()
p = opti.parameter()
opti.set_value(p, 3.0)

opti.minimize((y - x**2)**2)
opti.subject_to(x**2 + y**2 == 1)
opti.subject_to(x + y >= 1)
opti.subject_to(opti.bounded(-2, x, 2))

opti.solver('ipopt', {'expand': True}, {'max_iter': 200})
sol = opti.solve()
print('x*=', sol.value(x), ' y*=', sol.value(y))

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        2
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        3

Total number of variables............................:        2
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        2
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        1
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  0.0000000e+00 1.00e+00 1.00e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## 7. QP Demo：qpsol（带插件 fallback）

我们解一个很小的凸二次规划：
- 变量 $x\in\mathbb{R}^2$
- 目标：最小化 $(x_0-1)^2+(x_1-2)^2$
- 约束：$x_0 + x_1 = 1$，并加上简单边界 $x\ge 0$

不同安装里 QP 插件可用性不一样：这里按 `qpoases → osqp → qrqp` 的顺序尝试，失败就自动换下一个。

In [9]:
# QP: min (x0-1)^2 + (x1-2)^2  s.t. x0+x1=1, x>=0
x = ca.SX.sym('x', 2)
f = (x[0] - 1)**2 + (x[1] - 2)**2
g = x[0] + x[1]
qp = {'x': x, 'f': f, 'g': g}

solver = None
used_plugin = None
for plugin in ['qpoases', 'osqp', 'qrqp']:
    try:
        opts = {}
        if plugin == 'qpoases':
            opts.update({'sparse': True, 'print_time': False})
        solver = ca.qpsol('solver', plugin, qp, opts)
        used_plugin = plugin
        break
    except Exception as e:
        print(f'plugin {plugin} not available: {e}')

if used_plugin is None:
    raise RuntimeError('No qpsol plugin available (tried qpoases/osqp/qrqp)')

sol = solver(
    lbx=[0, 0], ubx=[ca.inf, ca.inf],
    lbg=1, ubg=1,
)
print('QP plugin:', used_plugin)
print('x* =', sol['x'])
print('f* =', sol['f'])


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.


qpOASES -- An Implementation of the Online Active Set Strategy.
Copyright (C) 2007-2015 by Hans Joachim Ferreau, Andreas Potschka,
Christian Kirches et al. All rights reserved.

qpOASES is distributed under the terms of the 
GNU Lesser General Public License 2.1 in the hope that it will be 
useful, but WITHOUT ANY WARRANTY; without even the implied warranty 
of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. 
See the GNU Lesser General Public License for more details.



####################   qpOASES  --  QP

## 8. Root-finding Demo：rootfinder（最小例子）

这里演示求解方程：
$$v^2 - a = 0$$
给定参数 $a=2$，用 `newton` 求 $\sqrt{2}$。

In [10]:
v = ca.SX.sym('v')
a = ca.SX.sym('a')
vfcn = ca.Function('vfcn', [v, a], [v**2 - a])

# rootfinder 的调用习惯：第一个输入是初值/猜测，后续是参数
rf = ca.rootfinder('rf', 'newton', vfcn)
sol_v = rf(1.0, 2.0)
print('sqrt(2) ~=', float(sol_v))

sqrt(2) ~= 1.4142135623730951
